In [5]:
import re # 正则表达式库
import collections # 词频统计库
import jieba # 结巴分词
from PIL import Image # 图像处理库
import matplotlib.pyplot as plt # 图像展示库
from matplotlib import colors
import csv
import pandas as pd
import jieba

In [2]:
pip install pytorch_pretrained_bert

Looking in indexes: http://mirrors.blockelite.cn:15019/pip/pypi/+simple/


In [4]:
jieba.case_sensitive = True # 可控制對於詞彙中的英文部分是否為case sensitive, 預設False


In [69]:
d = pd.read_csv("Updated_Linlan_全平台_reply_8_14.csv")
df = df.drop_duplicates(subset=None, keep='first', inplace=False)


In [7]:
# 去除字母数字、表情和其他字符
def clear_nonchinese_character(sentence):
    if not isinstance(sentence, str):  # 检查是否为字符串类型
        pass
    else:
        pat1 = '[a-zA-Z0-9]'
        pat2 = re.compile('[^\s1234567890:：\u4e00-\u9fa5]+')
        pat3 = '[’!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]+'
        #sentence = re.sub(pat1, '', sentence)  # 去除英文字母和数字
        sentence = re.sub(pat2, '', sentence)  # 去除表情和其他字符
        sentence = re.sub(pat3, '', sentence)  # 去除残留的冒号和其他符号
        sentence = ''.join(sentence.split())  # 去除空白
    return str(sentence)
df["content"]=df["content"].apply(clear_nonchinese_character)

In [16]:
df = df.dropna(subset=['content'])

In [8]:
# 判断字符串是否包含中文字符
def is_chinese(s):
    return bool(re.search('[\u4e00-\u9fff]', s))

#删掉空白字段和没有中文字符的字段
mask = df["content"].isnull() | df["content"].apply(lambda x: not is_chinese(str(x)))
dataset=df[~mask]
dataset

,floor,user_name,user_id,publish_time,content,gp_count,bp_count
0,1,巴哈小幫手,fuligirl,2023-08-08 12:00:01,由星重力數位科技代理的本格派戰棋玩法的像素風手機遊戲鈴蘭之劍：為這和平的世界將於今日正式上市...,17,0
2,5,星支星哞斬,coo741456,2023-08-08 20:14:21,20232921互相幫忙點感謝,0,0
3,6,Jacky,j8144123,2023-08-08 20:22:29,20233互相建設,0,0
4,7,炙焰亞卓,CDenny,2023-08-08 20:32:49,20232556互助互助,0,0
5,8,Vicious,kuso326,2023-08-08 20:58:49,202331互相幫助,0,0
...,...,...,...,...,...,...,...
10931,NaN,NaN,NaN,NaN,非常好玩,NaN,NaN
10932,NaN,NaN,NaN,NaN,下載31就不能下載遊戲做好在放上來好嗎,NaN,NaN
10933,NaN,NaN,NaN,NaN,封測公測大家的建議都沒在聽只是想噱錢吃吃睡吧,NaN,NaN
10934,NaN,NaN,NaN,NaN,靠这一款了,NaN,NaN


In [70]:
import time
import math
import pandas as pd
import numpy as np
import torch
from torch import nn
from tqdm import tqdm
from sklearn.metrics import classification_report
from concurrent.futures import ThreadPoolExecutor
from torch.utils.data import TensorDataset, DataLoader
from pytorch_pretrained_bert import BertTokenizer, BertModel
from pytorch_pretrained_bert.optimization import BertAdam


class ClassifyModel(nn.Module):
    def __init__(self, pretrained_model_name_or_path, num_labels, is_lock=False):
        super(ClassifyModel, self).__init__()
        self.bert = BertModel.from_pretrained(pretrained_model_name_or_path)
        config = self.bert.config
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(768, num_labels)
        if is_lock:
            # 加载并冻结bert模型参数
            for name, param in self.bert.named_parameters():
                if name.startswith('pooler'):
                    continue
                else:
                    param.requires_grad_(False)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None):
        _, pooled = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        pooled = self.dropout(pooled)
        logits = self.classifier(pooled)
        # 测试1，直接
        return logits


class DataProcessForSingleSentence(object):
    def __init__(self, bert_tokenizer, max_workers=10):
        """
        :param bert_tokenizer: 分词器
        :param max_workers:  包含列名comment和sentiment的data frame
        """
        self.bert_tokenizer = bert_tokenizer
        self.pool = ThreadPoolExecutor(max_workers=max_workers)

    def get_input(self, dataset, max_seq_len=30):
        sentences = dataset["review"].tolist()
        labels = dataset["label"].tolist()
        # 切词
        token_seq = list(self.pool.map(self.bert_tokenizer.tokenize, sentences))
        # 获取定长序列及其mask
        result = list(self.pool.map(self.trunate_and_pad, token_seq,
                                    [max_seq_len] * len(token_seq)))
        seqs = [i[0] for i in result]
        seq_masks = [i[1] for i in result]
        seq_segments = [i[2] for i in result]

        t_seqs = torch.tensor(seqs, dtype=torch.long)
        t_seq_masks = torch.tensor(seq_masks, dtype=torch.long)
        t_seq_segments = torch.tensor(seq_segments, dtype=torch.long)
        t_labels = torch.tensor(labels, dtype=torch.long)

        return TensorDataset(t_seqs, t_seq_masks, t_seq_segments, t_labels)

    def trunate_and_pad(self, seq, max_seq_len):
        # 对超长序列进行截断
        if len(seq) > (max_seq_len - 2):
            seq = seq[0: (max_seq_len - 2)]
            # 添加特殊字符
        seq = ['[CLS]'] + seq + ['[SEP]']
        # id化
        seq = self.bert_tokenizer.convert_tokens_to_ids(seq)
        # 根据max_seq_len与seq的长度产生填充序列
        padding = [0] * (max_seq_len - len(seq))
        # 创建seq_mask
        seq_mask = [1] * len(seq) + padding
        # 创建seq_segment
        seq_segment = [0] * len(seq) + padding
        # 对seq拼接填充序列
        seq += padding
        assert len(seq) == max_seq_len
        assert len(seq_mask) == max_seq_len
        assert len(seq_segment) == max_seq_len
        return seq, seq_mask, seq_segment


def load_data(filepath, pretrained_model_name_or_path, max_seq_len, batch_size):
    """
    加载excel文件，有train和test 的sheet
    :param filepath: 文件路径
    :param pretrained_model_name_or_path: 使用什么样的bert模型
    :param max_seq_len: bert最大尺寸，不能超过512
    :param batch_size: 小批量训练的数据
    :return: 返回训练和测试数据迭代器 DataLoader形式
    """
    df=pd.read_csv(filepath,encoding='utf-8')
    X=df["review"]
    y=df["label"]
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # 创建新的 DataFrame 来存储训练集和测试集
    raw_train_data = pd.DataFrame({'review': X_train, 'label': y_train})
    raw_test_data = pd.DataFrame({'review': X_test, 'label': y_test})
    
    # 分词工具
    bert_tokenizer = BertTokenizer.from_pretrained(pretrained_model_name_or_path, do_lower_case=True)
    processor = DataProcessForSingleSentence(bert_tokenizer=bert_tokenizer)
    # 产生输入句 数据
    train_data = processor.get_input(raw_train_data, max_seq_len)
    test_data = processor.get_input(raw_test_data, max_seq_len)

    train_iter = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
    test_iter = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)
    # 训练和测试的样本数量
    total_train_batch = math.ceil(len(raw_train_data) / batch_size)
    total_test_batch = math.ceil(len(raw_test_data) / batch_size)
    return train_iter, test_iter, total_train_batch, total_test_batch


def evaluate_accuracy(data_iter, net, device, batch_count):
    # 记录预测标签和真实标签
    prediction_labels, true_labels = [], []
    with torch.no_grad():
        for batch_data in tqdm(data_iter, desc='eval', total=batch_count):
            batch_data = tuple(t.to(device) for t in batch_data)
            # 获取给定的输出和模型给的输出
            labels = batch_data[-1]
            output = net(*batch_data[:-1])
            predictions = output.softmax(dim=1).argmax(dim=1)
            prediction_labels.append(predictions.detach().cpu().numpy())
            true_labels.append(labels.detach().cpu().numpy())

    return classification_report(np.concatenate(true_labels), np.concatenate(prediction_labels))


if __name__ == '__main__':
    batch_size, max_seq_len = 32, 200
    train_iter, test_iter, train_batch_count, test_batch_count = load_data('weibo_senti_100k.csv', 'bert-base-chinese', max_seq_len, batch_size)
    # 加载模型
    # model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=2)
    model = ClassifyModel('bert-base-chinese', num_labels=2, is_lock=True)
    print(model)

    optimizer = BertAdam(model.parameters(), lr=5e-05)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    loss_func = nn.CrossEntropyLoss()

    for epoch in range(4):
        start = time.time()
        model.train()
        # loss和精确度
        train_loss_sum, train_acc_sum, n = 0.0, 0.0, 0
        for step, batch_data in tqdm(enumerate(train_iter), desc='train epoch:{}/{}'.format(epoch + 1, 4)
                                    , total=train_batch_count):
            batch_data = tuple(t.to(device) for t in batch_data)
            batch_seqs, batch_seq_masks, batch_seq_segments, batch_labels = batch_data

            logits = model(batch_seqs, batch_seq_masks, batch_seq_segments)
            loss = loss_func(logits, batch_labels)
            loss.backward()
            train_loss_sum += loss.item()
            logits = logits.softmax(dim=1)
            train_acc_sum += (logits.argmax(dim=1) == batch_labels).sum().item()
            n += batch_labels.shape[0]
            optimizer.step()
            optimizer.zero_grad()
        # 每一代都判断
        model.eval()

        result = evaluate_accuracy(test_iter, model, device,test_batch_count)
        print('epoch %d, loss %.4f, train acc %.3f, time: %.3f' %
              (epoch + 1, train_loss_sum / n, train_acc_sum / n, (time.time() - start)))
        print(result)

    torch.save(model, 'fine_tuned_chinese_bert.bin')

t_total value of -1 results in schedule not being applied


ClassifyModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
           

train epoch:1/4:  17%|█▋        | 456/2625 [22:00<1:44:40,  2.90s/it]


KeyboardInterrupt: 

In [9]:
def process_data(sentences, max_seq_len, pretrained_model_name_or_path):
    bert_tokenizer = BertTokenizer.from_pretrained(pretrained_model_name_or_path, do_lower_case=True)
    processor = DataProcessForSingleSentence(bert_tokenizer=bert_tokenizer)
    
    token_seq = list(processor.pool.map(processor.bert_tokenizer.tokenize, sentences))
    result = list(processor.pool.map(processor.trunate_and_pad, token_seq, [max_seq_len] * len(token_seq)))
    seqs = [i[0] for i in result]
    seq_masks = [i[1] for i in result]
    seq_segments = [i[2] for i in result]
    
    t_seqs = torch.tensor(seqs, dtype=torch.long)
    t_seq_masks = torch.tensor(seq_masks, dtype=torch.long)
    t_seq_segments = torch.tensor(seq_segments, dtype=torch.long)
    
    return TensorDataset(t_seqs, t_seq_masks, t_seq_segments)

def predict(dataset, model, device, max_seq_len, pretrained_model_name_or_path):
    model.eval()
    contents = dataset["content"].tolist()
    data = process_data(contents, max_seq_len, pretrained_model_name_or_path)
    data_iter = DataLoader(data, batch_size=32)
    
    prediction_labels = []
    with torch.no_grad():
        for batch_data in data_iter:
            batch_data = tuple(t.to(device) for t in batch_data)
            output = model(*batch_data)
            predictions = output.softmax(dim=1).argmax(dim=1)
            prediction_labels.extend(predictions.detach().cpu().numpy())
    
    return prediction_labels


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.load('fine_tuned_chinese_bert.bin')
model.to(device)

predictions = predict(dataset, model, device, max_seq_len=200, pretrained_model_name_or_path='bert-base-chinese')

dataset['prediction'] = predictions
dataset.to_csv('predicted_dataset2.csv', index=False)

/var/folders/jj/np9v4hjs1jn8f0nc2j4_dsdh0000gn/T/ipykernel_94351/1183884867.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['prediction'] = predictions


In [10]:
data=pd.read_csv("predicted_dataset2.csv")

In [17]:
data[data["prediction"]==1]["content"].head(50)

0     由星重力數位科技代理的本格派戰棋玩法的像素風手機遊戲鈴蘭之劍：為這和平的世界將於今日正式上市...
1                                       20232921互相幫忙點感謝
2                                             20233互相建設
3                                          20232556互助互助
4                                            202331互相幫助
5                                   20231護幫點完後留言會回點還有2次
6                              還沒建設完我的可以點雙方互相都有獎勵202351
7                                   20231316幫點完留言我再回點謝你
8     不管你是新手或是老手希望大家來到哈啦區都能夠感受到輕鬆無壓力的氛圍但請務必記住輕鬆並不等於放...
9                                      2023521今日幫助留言會互點
10                                            2023941幫忙
13                                       不要沉下去一起迎向美好的未來
14                   序號可能會沒領到但這個獎勵是實打實的推廣一下期望每個人都到達白金光輝
16    自推讓更多人看到把對面推平都有10榮耀防守被打爆目前看起來是不扣榮耀每天五場一場10分一天5...
19                             官方號感覺超多的那個什麼好友的巴哈幫別人點建設的
20                                              810晚上推廣
22                                                  推一個
23                                              

In [39]:
lines1=dataset[(dataset['prediction']==1)]

pos_df = pd.concat([lines1])
print(pos_df['content'])



0        由星重力數位科技代理的本格派戰棋玩法的像素風手機遊戲鈴蘭之劍：為這和平的世界將於今日正式上市...
2                                          20232921互相幫忙點感謝
3                                                20233互相建設
4                                             20232556互助互助
5                                               202331互相幫助
                               ...                        
10928                                   開局送7抽保底200抽機率3見好就收
10930                                                終於等到了
10931                                                 非常好玩
10934                                                靠这一款了
10935                                                 女角色很
Name: content, Length: 5714, dtype: object


In [41]:
lines2 = dataset[(dataset['prediction']==0)]
neg_df = pd.concat([lines2])
posl=pos_df['content'].tolist()
negl=neg_df['content'].tolist()
dataset["Classification"]=dataset['content'].apply(lambda x: "好评" if x in posl else "差评")

/var/folders/jj/np9v4hjs1jn8f0nc2j4_dsdh0000gn/T/ipykernel_94351/3134384460.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["Classification"]=dataset['content'].apply(lambda x: "好评" if x in posl else "差评")


In [42]:
dataset.to_excel("中间表.xlsx", index=False)

In [55]:
#对评论分词
jieba.load_userdict('dict.txt' ) #启用自定义词典
with open('stopwords.txt') as f: #启用停用词表，过滤掉标点符号和一些与情感无关的词
   stop_word = f.read()

# 去除字母数字、表情和其他字符
def clear_nonchinese_character(sentence):
    if not isinstance(sentence, str):  # 检查是否为字符串类型
        pass
    else:
        pat1 = '[0-9]'
        pat2 = re.compile('[^\s1234567890:：\u4e00-\u9fa5]+')
        pat3 = '[’!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]+'
        sentence = re.sub(pat1, '', sentence)  # 去除英文字母和数字
        sentence = re.sub(pat2, '', sentence)  # 去除表情和其他字符
        sentence = re.sub(pat3, '', sentence)  # 去除残留的冒号和其他符号
        sentence = ''.join(sentence.split())  # 去除空白
    return str(sentence)

def cuttxt(x):
    cut=jieba.cut(clear_nonchinese_character(x))
    result=[k for k in cut if k not in stop_word]
    return " ".join(result)

In [56]:
dataset["分词"]=dataset["content"].apply(cuttxt)

/var/folders/jj/np9v4hjs1jn8f0nc2j4_dsdh0000gn/T/ipykernel_94351/3296800223.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["分词"]=dataset["content"].apply(cuttxt)


In [57]:
# 判断字符串是否包含中文字符
def is_chinese(s):
    return bool(re.search('[\u4e00-\u9fff]', s))

#删掉空白字段和没有中文字符的字段
mask = dataset["分词"].isnull() | dataset["分词"].apply(lambda x: not is_chinese(str(x)))
dataset=dataset[~mask]
dataset.head()

,floor,user_name,user_id,publish_time,content,gp_count,bp_count,prediction,Classification,分词,分类
0,1,巴哈小幫手,fuligirl,2023-08-08 12:00:01,由星重力數位科技代理的本格派戰棋玩法的像素風手機遊戲鈴蘭之劍：為這和平的世界將於今日正式上市...,17,0,1,好评,由星 重力 數位 科技 代理 本格派 戰棋 玩法 像素風 手機 遊戲 和平 世界 將 於 今...,推广活动
2,5,星支星哞斬,coo741456,2023-08-08 20:14:21,20232921互相幫忙點感謝,0,0,1,好评,幫忙 點 感謝,None
3,6,Jacky,j8144123,2023-08-08 20:22:29,20233互相建設,0,0,1,好评,建設,None
4,7,炙焰亞卓,CDenny,2023-08-08 20:32:49,20232556互助互助,0,0,1,好评,互助 互助,推广活动
5,8,Vicious,kuso326,2023-08-08 20:58:49,202331互相幫助,0,0,1,好评,幫助,None


In [58]:
cidai=[]
for n in dataset['分词']:
    word_list = n.split()
    for i in word_list:
        cidai.append(i)



In [60]:

def classify(string):
    c=[]
    keywords_list=[]
    keywords_list.append(r'(像素|画面)')
    keywords_list.append(r'(英雄|角色|麦莎|拉维耶|伊利亚|世界观|剧情|加速|主线|武器|装备)')
    keywords_list.append(r'(关卡|战术|玩法|战棋|战斗|阵营|体力)')
    keywords_list.append(r'(更新|福利|活动|礼包|模式|奖励|新手|氪金|课金|晶石|保底|运营|官方|营销)')
    keywords_list.append(r'(bug|问题|卡|网络|稳定|服务器)')
    keywords_list.append(r'(推|转|送|感谢|转发|点赞|互助|帮助|帮忙|点)')
    keywords_list.append(r'(卡|卡池|抽|抽中|抽不中|非洲|欧洲|非酋|碎片)')
    keywords_list.append(r'(最终幻想|FFT|fft|梦幻模拟战|皇家骑士团)')
    #word_list = string.split()
    for i in range(len(keywords_list)):
        c.append(re.findall(keywords_list[i], string))
    
    max_counter = max(c, key=len)
    if max_counter == 0:
        return '其他'
    if max_counter and max_counter == c[0]:
        return '游戏风格'
    if max_counter and max_counter == c[1]:
        return '游戏内容'
    if max_counter and max_counter == c[2]:
        return '局内体验'
    if max_counter and max_counter == c[3]:
        return '消费体验'
    if max_counter and max_counter == c[4]:
        return 'bug'
    if max_counter and max_counter == c[5]:
        return '推广活动'
    if max_counter and max_counter == c[6]:
        return '抽卡体验'
    if max_counter and max_counter == c[7]:
        return '同类竞品'


dataset['分类']=dataset['content'].apply(classify)
dataset.dropna(axis=0, how='any', subset=['分类'])

,floor,user_name,user_id,publish_time,content,gp_count,bp_count,prediction,Classification,分词,分类
0,1,巴哈小幫手,fuligirl,2023-08-08 12:00:01,由星重力數位科技代理的本格派戰棋玩法的像素風手機遊戲鈴蘭之劍：為這和平的世界將於今日正式上市...,17,0,1,好评,由星 重力 數位 科技 代理 本格派 戰棋 玩法 像素風 手機 遊戲 和平 世界 將 於 今...,推广活动
4,7,炙焰亞卓,CDenny,2023-08-08 20:32:49,20232556互助互助,0,0,1,好评,互助 互助,推广活动
17,1,cpan,cpan,2009-03-09 11:29:41,不管你是新手或是老手希望大家來到哈啦區都能夠感受到輕鬆無壓力的氛圍但請務必記住輕鬆並不等於放...,1000,500,1,好评,新手 老手 來到 哈啦區 能夠 感受到 輕鬆 無 壓力 氛圍 務必 記住 輕鬆 並 等於 放...,消费体验
37,1,桐,play24621681,2023-08-10 11:53:38,有人知道榮譽之戰防守失敗不知道會不會掉榮耀值嗎我個人覺得應該不會他也沒說破壞到怎樣的程度算贏...,72,0,0,差评,榮譽 戰 防守 失敗 會不會 掉 榮耀 嗎 個 不會 說 破壞 怎樣 程度 算贏 不會 擺爛...,推广活动
42,6,桐,play24621681,2023-08-10 14:19:08,序號可能會沒領到但這個獎勵是實打實的推廣一下期望每個人都到達白金光輝,1,0,1,好评,序號 會 領到 個 獎勵 是實 打實 推廣 期望 個 到達 白金 光輝,推广活动
...,...,...,...,...,...,...,...,...,...,...,...
10917,NaN,NaN,NaN,NaN,糞2保底200抽一萬五台幣,NaN,NaN,1,好评,糞 保底 萬五 台幣,消费体验
10919,NaN,NaN,NaN,NaN,無法更新,NaN,NaN,1,好评,無法 更新,消费体验
10926,NaN,NaN,NaN,NaN,到底是有多摳啊前期只能抽1次10連馬的新手關卡打的就很累個人評價就是個開發者不關遊戲體驗只想...,NaN,NaN,0,差评,摳 前期 連 馬 新手 關卡 個 評價 個 開發 不關 遊戲 體驗 想 賺錢 爛 遊戲,抽卡体验
10928,NaN,NaN,NaN,NaN,開局送7抽保底200抽機率3見好就收,NaN,NaN,1,好评,開局 送 保底 機率 見好 收,抽卡体验


In [68]:
dataset.to_excel("原始表.xlsx")

In [61]:
pos_tiyan=dataset[(dataset["Classification"]=="好评")]
#TF-IDF算法提取正向评论关键词
import jieba.analyse

pos1=[]
for n in pos_tiyan['分词']:
    word_list = n.split()
    for i in word_list:
        if i != "游戏":
            pos1.append(i)
        
text=" ".join(pos1)
tags = pd.DataFrame(jieba.analyse.extract_tags(text,topK=100,withWeight=True), columns=['词语', '重要值'])
print(tags)

tags.to_excel("正向最高词频.xlsx")

    词语       重要值
0   遊戲  0.242971
1   活動  0.143383
2   序號  0.095988
3   分享  0.084333
4   關於  0.076191
..  ..       ...
95  關卡  0.017998
96  打法  0.017948
97  輕鬆  0.017398
98  王女  0.017398
99  抽到  0.017398

[100 rows x 2 columns]


In [62]:

neg_nr=dataset[(dataset['分类']=='游戏内容') & (dataset["Classification"]=="差评")]

import jieba.analyse

neg1=[]
for n in neg_nr['分词']:
    word_list = n.split()
    for i in word_list:
        if i != "游戏":
            neg1.append(i)
        
text=" ".join(neg1)
tags = pd.DataFrame(jieba.analyse.extract_tags(text,topK=100,withWeight=True), columns=['词语', '重要值'])
print(tags)

tags.to_excel("负向最高词频1.xlsx")

    词语       重要值
0   角色  0.329499
1   技能  0.142584
2   玩家  0.124055
3   遊戲  0.121141
4   任務  0.052012
..  ..       ...
95  幻想  0.013029
96  碎片  0.012656
97  保底  0.012555
98  搭配  0.012512
99  貝拉  0.012509

[100 rows x 2 columns]


In [63]:
neg_nr=dataset[(dataset['分类']=='消费体验') & (dataset["Classification"]=="差评")]

import jieba.analyse

neg1=[]
for n in neg_nr['分词']:
    word_list = n.split()
    for i in word_list:
        if i != "游戏":
            neg1.append(i)
        
text=" ".join(neg1)
tags = pd.DataFrame(jieba.analyse.extract_tags(text,topK=100,withWeight=True), columns=['词语', '重要值'])
print(tags)

tags.to_excel("负向最高词频2.xlsx")

    词语       重要值
0   遊戲  0.347533
1   玩家  0.190499
2   官方  0.155984
3   保底  0.153016
4   氪金  0.145683
..  ..       ...
95  比較  0.014042
96  關卡  0.014042
97  手機  0.014042
98  遊玩  0.014042
99  營運  0.014042

[100 rows x 2 columns]


In [64]:
neg_nr=dataset[(dataset['分类']=='局内体验') & (dataset["Classification"]=="差评")]

import jieba.analyse

neg1=[]
for n in neg_nr['分词']:
    word_list = n.split()
    for i in word_list:
        if i != "游戏":
            neg1.append(i)
        
text=" ".join(neg1)
tags = pd.DataFrame(jieba.analyse.extract_tags(text,topK=100,withWeight=True), columns=['词语', '重要值'])
print(tags)

tags.to_excel("负向最高词频3.xlsx")

    词语       重要值
0   玩法  0.296783
1   玩家  0.257763
2   技能  0.140875
3   手游  0.130763
4   氪金  0.085499
..  ..       ...
95  能力  0.014534
96  站位  0.014518
97  要素  0.014340
98  特定  0.014307
99  肯定  0.014129

[100 rows x 2 columns]


In [65]:
neg_nr=dataset[(dataset['分类']=='抽卡体验') & (dataset["Classification"]=="差评")]

import jieba.analyse

neg1=[]
for n in neg_nr['分词']:
    word_list = n.split()
    for i in word_list:
        if i != "游戏":
            neg1.append(i)
        
text=" ".join(neg1)
tags = pd.DataFrame(jieba.analyse.extract_tags(text,topK=100,withWeight=True), columns=['词语', '重要值'])
print(tags)

tags.to_excel("负向最高词频4.xlsx")

    词语       重要值
0   遊戲  0.274192
1   角色  0.117102
2   關卡  0.098709
3   玩家  0.097251
4   保底  0.079678
..  ..       ...
95  慢慢  0.013803
96  紅隼  0.013548
97  主線  0.013548
98  東西  0.013548
99  繼續  0.013548

[100 rows x 2 columns]


In [67]:

#重合度test
p1 = r'玩家'
p2 = r'劇情'
p3 = r'體力|时间'
p4 = r'玩法'
p5 = r'人物|角色|英雄'
p6 = r'官方'
p7 = r'氪金'
p8 = r'福利|獎勵|保底'
p9 = r'技能|武器|碎片'
p10 = r'關卡'

p_list = [p1, p2, p3, p4, p5, p6, p7,p8,p9,p10]

def Relation(a, b):
    sample = df[df['content'].str.contains(a, na=False)]  # Drop NaN values before using str.contains
    sample['relation'] = sample['content'].apply(lambda x: 1 if re.search(b, x) else 0)
    output = sample['relation'].sum() / sample['relation'].count()
    return output

import numpy as np
import pandas as pd
import re

# Assuming df is defined somewhere
result = pd.DataFrame(np.nan, index=p_list, columns=p_list)

for p in p_list:
    for i in p_list:
        result.loc[p, i] = Relation(p, i)

print(result)
result.to_csv('result.csv')


/var/folders/jj/np9v4hjs1jn8f0nc2j4_dsdh0000gn/T/ipykernel_94351/1132550375.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['relation'] = sample['content'].apply(lambda x: 1 if re.search(b, x) else 0)
/var/folders/jj/np9v4hjs1jn8f0nc2j4_dsdh0000gn/T/ipykernel_94351/1132550375.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['relation'] = sample['content'].apply(lambda x: 1 if re.search(b, x) else 0)
/var/folders/jj/np9v4hjs1jn8f0nc2j4_dsdh0000gn/T/ipykernel_94351/1132550375.py:17: Set

                玩家        劇情     體力|时间        玩法  人物|角色|英雄        官方  \
玩家        1.000000  0.098990  0.181818  0.137374  0.400000  0.141414   
劇情        0.192913  1.000000  0.141732  0.102362  0.259843  0.074803   
體力|时间     0.301003  0.120401  1.000000  0.130435  0.361204  0.080268   
玩法        0.304933  0.116592  0.174888  1.000000  0.394619  0.112108   
人物|角色|英雄  0.254172  0.084724  0.138639  0.112965  1.000000  0.070603   
官方        0.308370  0.083700  0.105727  0.110132  0.242291  1.000000   
氪金        0.368056  0.000000  0.173611  0.173611  0.416667  0.076389   
福利|獎勵|保底  0.152350  0.085900  0.102107  0.059968  0.197731  0.077796   
技能|武器|碎片  0.176364  0.083636  0.132727  0.083636  0.420000  0.047273   
關卡        0.152284  0.126904  0.131980  0.045685  0.258883  0.055838   

                氪金  福利|獎勵|保底  技能|武器|碎片        關卡  
玩家        0.107071  0.189899  0.195960  0.060606  
劇情        0.000000  0.208661  0.181102  0.098425  
體力|时间     0.083612  0.210702  0.244147  0.086957  
玩法 

/var/folders/jj/np9v4hjs1jn8f0nc2j4_dsdh0000gn/T/ipykernel_94351/1132550375.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['relation'] = sample['content'].apply(lambda x: 1 if re.search(b, x) else 0)
/var/folders/jj/np9v4hjs1jn8f0nc2j4_dsdh0000gn/T/ipykernel_94351/1132550375.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['relation'] = sample['content'].apply(lambda x: 1 if re.search(b, x) else 0)
/var/folders/jj/np9v4hjs1jn8f0nc2j4_dsdh0000gn/T/ipykernel_94351/1132550375.py:17: Set